### 1. Import packages

In [4]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from langdetect import detect

In [31]:
#!pip install pymorphy2
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

### 2. Functions

#### Merge data

In [1]:
import os

In [3]:
def concat_data(folder):
    files = os.listdir('data/'+str(folder))
    df = pd.read_csv('data/'+str(folder)+'/'+str(files[0]))
    for i in range(len(files)):
        df2 = pd.read_csv('data/'+str(folder)+'/'+str(files[i]))
        df = pd.concat([df,df2])
    print(df.shape)
    print('/...Export the data')
    df.to_csv('./data/'+str(folder)+'/'+str(folder)+'_full')

In [ ]:
#give the function the name of the folder where the samples stored
concat_data('rewards')

In [16]:
df1 = pd.read_csv('./data/projects/projects_1_800.csv')
df2 = pd.read_csv('./data/projects/projects_801_1600.csv')

In [17]:
projects = pd.concat([df1,df2])

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [18]:
projects.to_csv('./data/projects/projects_full.csv')

#### Clean projects

In [ ]:
#upload data
projects = pd.read_csv('./data/projects/projects_full.csv')
creaters = pd.read_csv('./data/creaters/creaters_full.csv')
rewards = pd.read_csv('./data/rewards/rewards_full.csv')

In [40]:
def tokenize(text):
    lang = {'de': 'dutch',
            'en': 'english',
            'fi': 'finnish',
            'fr': 'french',
            'no': 'norwegian'}
    words = text.lower().split()
    words_normalized = []
    language = detect(text)
    if language in lang.keys():
        words = [word for word in words if word not in stopwords.words(lang[language])]

    for w in words:
        lemma = pymorphy2_analyzer.parse(w)
        words_normalized.append(lemma[0].normal_form)
    return ', '.join(words_normalized)

In [ ]:
#reset_index
projects.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], inplace=True)
creaters.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], inplace=True)
rewards.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1'], inplace=True)

In [41]:
#run to clean the data
projects['funded'] = projects['funded'].apply(lambda x: ''.join(re.findall(r'\d', x))).astype('float64')
projects['target'] = projects['target'].apply(lambda x: ''.join(re.findall(r'\d', x))).astype('float64')
projects['end_date'] = pd.to_datetime(projects['end_date'])
projects['short_des'] = projects['short_des'].apply(lambda x: tokenize(str(x)))

In [ ]:
#export data
projects.to_csv('./data/projects/projects_cleaned.csv')
rewards.to_csv('./data/rewards/rewards_cleaned.csv')
creaters.to_csv('./data/creaters/creaters_cleaned.csv')

In [43]:
projects.head(10)

Unnamed: 0  ...                                          short_des
0           0  ...  participez, naissance, cassava, réunion!, mari...
1           1  ...  sauvons, ensemble, "ibiza, horse, valley",, sa...
2           2  ...     soutenez, le, projet, sur, kisskissbankbank, !
3           3  ...  collectif, d’artistes,, playschool, a, besoin,...
4           4  ...       164, pages,, 2, couvertures, 0, publicité, !
5           5  ...  soutenez, le, tournage, du, teaser, du, long-m...
6           6  ...  pré-réservez, aujourd’hui, prochaine, micro-im...
7           7  ...  soutenez, création, route, 64,, nouveau, média...
8           8  ...  bac, litière, connecté,, esthétique, performan...
9           9  ...  soutenez-nous, remettons, ensemble, goût, jour...

[10 rows x 20 columns]